# Player Detection Model Local Training

This notebook trains a YOLOv8x model for football player detection using your local GPU.

In [1]:
# Install required packages
!pip install ultralytics
!pip install roboflow


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 60.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.1 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Import required packages
from roboflow import Roboflow
from ultralytics import YOLO
import torch
from pathlib import Path

In [3]:
# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cuda":
    print(f"GPU Information: {torch.cuda.get_device_name(0)}")
    print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("No GPU detected. Training will be slower on CPU.")

Using device: cuda
GPU Information: NVIDIA RTX A6000
Available GPU memory: 51.04 GB


## Download Dataset

Download the football players detection dataset from Roboflow.

In [4]:
import roboflow

rf = roboflow.Roboflow(api_key="yiIi6EUhjEeqSzZT7xAE")

# List all projects for your workspace
workspace = rf.workspace()

print(workspace)

loading Roboflow workspace...
{
  "name": "College",
  "url": "college-vxiyc",
  "projects": [
    "college-vxiyc/football-players-detection-3zvbc-ojf1h"
  ]
}


In [ ]:

try:
    # Download dataset from Roboflow (using the same project as in Kaggle)
    print("Downloading dataset from Roboflow...")
    rf = roboflow.Roboflow(api_key="yiIi6EUhjEeqSzZT7xAE")
    print("Connected to Roboflow")
    project = rf.workspace("college-vxiyc").project("football-players-detection-3zvbc-ojf1h")
    print(f"Found project: {project.name}")
    version = project.version(1)
    print(f"Using version: {version.id}")
    dataset = version.download("yolov8", location="player-detection-dataset")
    print("Dataset downloaded successfully")
except Exception as e:
    print(f"Error downloading dataset: {e}")

Connected to Roboflow
loading Roboflow workspace...
loading Roboflow project...
Found project: football-players-detection
Using version: college-vxiyc/football-players-detection-3zvbc-ojf1h/1



Extracting Dataset Version Zip to dataset in yolov8:: 100%|██████████| 756/756 [00:00<00:00, 5955.05it/s]

Dataset downloaded successfully


In [6]:
# check the dataset variable
print(dataset)
print("Dataset details:")
print(f"Name: {dataset.name}")
print(f"Version: {dataset.version}")
print(f"Location: {dataset.location}")


Dataset details:
Name: football-players-detection
Version: 1
Location: /home/sidhartha22499/Desktop/IRRA/foot/dataset


## Train the Model

Initialize and train the YOLOv8x model on the football players detection dataset.

In [7]:
data_yaml_path = Path(dataset.location) / "data.yaml"
models_dir = Path("models")
if not models_dir.exists():
    models_dir.mkdir(parents=True, exist_ok=True)
# Initialize a YOLOv8x model
print("Initializing YOLOv8x model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")    
model = YOLO('yolov8x.pt')

# Start training
print("Starting model training...")
results = model.train(
    data=str(data_yaml_path),
    epochs=100,
    imgsz=1280,
    batch=4,
    device = device,
    project=str(models_dir),
    name="football_detector"
)

Initializing YOLOv8x model...
Using device: cuda


100%|██████████| 131M/131M [00:02<00:00, 50.6MB/s] 


Starting model training...
Ultralytics 8.3.76 🚀 Python-3.8.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/home/sidhartha22499/Desktop/IRRA/foot/dataset/data.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=1280, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=models, name=football_detector, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False,

100%|██████████| 5.35M/5.35M [00:00<00:00, 14.3MB/s]


AMP: checks passed ✅


...

    100/100      12.3G     0.5153      0.244     0.8129         49       1280: 100%|██████████| 75/75 [00:24<00:00,  3.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.86it/s]

                   all         49       1174       0.95      0.871      0.902      0.653



100 epochs completed in 0.818 hours.
Optimizer stripped from models/football_detector/weights/last.pt, 136.9MB
Optimizer stripped from models/football_detector/weights/best.pt, 136.9MB

Validating models/football_detector/weights/best.pt...
Ultralytics 8.3.76 🚀 Python-3.8.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
Model summary (fused): 112 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.53it/s]


                   all         49       1174      0.924      0.887      0.919      0.671
                  ball         45         45      0.894      0.644      0.747      0.366
            goalkeeper         38         39      0.878      0.949      0.951      0.738
                player         49        973      0.978      0.989      0.994      0.839
               referee         49        117      0.948      0.966      0.985      0.741
Speed: 0.3ms preprocess, 24.7ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to models/football_detector


## Validate the Model

Validate the trained model on the validation dataset.

In [8]:
# Validate the model
print("Validating the model...")
best_model_path = models_dir / "football_detector" / "weights" / "best.pt"
val_results = model.val(data=str(data_yaml_path), imgsz=1280)

print(f"Training complete. Best model saved at: {best_model_path}")
print(f"Validation results: mAP50-95={val_results.box.map:.3f}, mAP50={val_results.box.map50:.3f}")

Validating the model...
Ultralytics 8.3.76 🚀 Python-3.8.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
Model summary (fused): 112 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /home/sidhartha22499/Desktop/IRRA/foot/dataset/valid/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  4.74it/s]


                   all         49       1174      0.916      0.881      0.912      0.681
                  ball         45         45      0.864      0.622      0.718       0.38
            goalkeeper         38         39      0.877      0.949       0.95      0.742
                player         49        973      0.977      0.988      0.994      0.854
               referee         49        117      0.948      0.966      0.984      0.748
Speed: 2.0ms preprocess, 41.4ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to models/football_detector2
Training complete. Best model saved at: models/football_detector/weights/best.pt
Validation results: mAP50-95=0.681, mAP50=0.912


## Test the Trained Model

You can run a test prediction on an image to verify the model works correctly.

In [ ]:
# Load the trained model
trained_model = YOLO(best_model_path)

# You can test the model on any football image
# If you have a test image, replace 'path/to/test/image.jpg' with the actual path
test_image = r"C:\Users\Sidhartha Garg\Desktop\football-analysis-CV-main\football-analysis-CV-main\foot\dataset\test\images\4b770a_3_6_png.rf.98a2dc73e60d2cb1ff9ffda743ec62ba.jpg"
results = trained_model(test_image)
results[0].show()  # Display the detection results


image 1/1 /home/sidhartha22499/Desktop/IRRA/foot/dataset/test/images/4b770a_1_6_png.rf.a7eccdca33d5c78376846f38bc004d27.jpg: 1280x1280 1 ball, 21 players, 3 referees, 39.1ms
Speed: 2.7ms preprocess, 39.1ms inference, 0.9ms postprocess per image at shape (1, 3, 1280, 1280)


Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)


## Update Configuration for Local Inference

Update your local configuration to use this new model.

In [10]:
# To use this model in your main_test.py,
# update the MODEL_SRC variable in config.py to point to your newly trained model
print(f"To use this model in your application, update the MODEL_SRC variable in config.py to: {best_model_path}")

To use this model in your application, update the MODEL_SRC variable in config.py to: models/football_detector/weights/best.pt
